## CIE4604 - Simulation and Visualisation
## Assignment 3 Exercise

#### Student Name(s) : 
#### Student Number(s): 
This script does a time series analysis of NDVI using the HANTS (Harmonic Analysis of Time Series) algorithm and visualize the results. HANTS is a software using the Fourier Analysis in order to detect outliers (clouds) and reconstruct image time series.  
The analysis is performed over Sector BXII, an agricultural area located close to Sevilla (Spain). 
Input data files for this script are:
* `NDVI_BXII_time_series_2017_2018_S2A_SP.tif  (Geotiff with NDVI time series) `
* `ts.mat  (Mat-file with day number information for HANTS) `

Optional (non-required) input file(s) are:

*` Training_2017_UTM30N_WGS84_SP.shp   (Training dataset) `

The actual HANTS algorithm is implemented in the Python function HANTS.py


<div class="alert alert-block alert-info">
This script is a template that you have to modify in places to obatin the desired results.

*See also HANTS.py*

</div>

In [ ]:
# Importing all the libraries required for this exercise
import numpy as np
import rasterio
from scipy.io import loadmat
import matplotlib.pyplot as plt
import matplotlib.colorbar as colorbar
from datetime import date, datetime
import matplotlib.colors as colors
import matplotlib.animation as animation
import shapefile 
import matplotlib as mpl

### Load NDVI for the area of interest
Input files:
* `NDVI_BXII_time_series_2017_2018_S2A_SP.tif - geotiff with NDVI` 
* `ts.mat - mat file with dates`

The following variables are created

* NDVI - 3D Matlab matrix with dimensions [lat lon time] , i.e. NDVI(:,:,k) contains the k'th NDVI image (time ts(k)) 
* ts   - Matlab array with the number of days since 12/04/2017

You should definitely take a look at each variable to see what it is about

In [ ]:
path = 'input/'
file_in = 'NDVI_BXII_time_series_2017_2018_S2A_SP.tif'

# read in file with geotiff geographic information
src = rasterio.open(path + file_in)

# print out metadata information
for k in src.meta:
  print(k,src.meta[k])

transform = src.transform
N = src.width
M = src.height
dx = transform.a
dy = transform.e
minx = transform.c
maxy = transform.f
if dy < 0:
    dy = -dy
# Read the image data, flip upside down if necessary
NDVI = src.read()
print(NDVI.shape)

In [ ]:
ndates = NDVI.shape[0]
ts = loadmat('input/ts.mat')['ts']
tsdates = (date.toordinal(date(2017, 4, 12)) + 366) + ts - 1
# Generate X and Y grid locations
x = minx + dx / 2 + dx * np.arange(N)
y = maxy - dy / 2 - dy * np.arange(M - 1, -1, -1)

## Make a scaled image plot of unfiltered NDVI for the first day
With the Matlab function imagesc you can make a color indexed image of the NDVI. Imagesc set by default the 'YDir' property to reverse, which is ok for matrices without x and y coordinates, but to obtain the proper  behaviour with x and y coordinate input 'YDir' has to be set to normal.

It is very important to select a good colormap for the NDVI. The default colormap for Matlab is |parula|, but this is not a very good choice for NDVI. Why? Have a look for instance at the following article  https://publiclab.org/notes/cfastie/08-26-2014/new-ndvi-colormap

The code for making the 2.5D is given below, but you should definitely replace the default Matlab colormap for something more sensible. See
also the instructions in Module 3.1.
In Python you can read the text file or `mat` file and load it using `matplotlib.colors.ListedColormap`
```python
data = scipy.io.loadmat('file.mat')['colors']
newcmp = colors.ListedColormap(data/255)
```
You can then declare this new colormap in the `imshow` function by declaring the input as `cmap = newcmp`


In [ ]:
# unfiltered NDVI data
extent = [x[0] / 1000, x[-1] / 1000, y[0] / 1000, y[-1] / 1000]
plt.figure(figsize=(8, 8))
index = 0
# SELECT APPROPRIATE COLORMAP FOR NDVI 
# colarmap(something)
# END COLORMAP SELECTION
    fig = plt.imshow(NDVI[index, :, :], extent=extent, cmap='something')
plt.title(
    'NDVI ' + datetime.fromordinal(tsdates[:, index] - 366).strftime('%Y-%m-%d') + ' DAY ' + str(int(ts[:, index])))

plt.clim(-1, 1)
plt.ylabel('North [km]')
plt.xlabel('East [km]')
plt.axis('image')
plt.show()

## Create a video with the unfiltered NDVI data
A video is created showing the unfiltered NDVI data. We use imagesc, colormap and colorbar to scale and display matrix NDVI(:,:,k) as an image for day, using a meaningful colormap of your choice (see previous section).
The figure statetement is called only once, imagesc is called from within the loop over days, and the output is grabbed by getframe for the video.

In [ ]:
frames = [] # for storing the generated images
# Using matplotlib qt in order to see the video
%matplotlib qt
fig, ax=plt.subplots()
for i in range(ndates):
#     im = Add your images here
#     title = Add your title here
    frames.append([im, title])
ani = animation.ArtistAnimation(fig, frames, interval=200, blit=False, repeat=False)
# Uncomment below line to save the video
ani.save('NDVI_unfiltered.gif')

## Compute reconstructed time series using HANTS
Inputs for HANTS are the times ts (days) and the NDVI time series for one pixel.
Other input parameters for HANTS are defined below:

In [ ]:
nb=365        # length of the base period measured in timeunits of ts
nf=3          # number of frequencies to be considered above the zero frequency
low=-1        # valid range minimum (values below are rejected right away)
high=1        # valid range maximum (values above are rejected right away)
fet=0.1       # fit error tolerance (points deviating more than fet from curve fit are rejected)
dod=1         # degree of overdeterminedness (iteration stops if number of points reaches the minimum required for curve fitting, plus dod). This is a safety measure
delta=0.1     # small positive number (e.g. 0.1) to suppress high amplitudes
HiLo='Lo'     # 2-character string indicating rejection of high or low outliers

Set the line and sample number of the pixel for which to apply HANTS.

We will show you later how to find the line and sample numbers for specific pixel areas, e.g. one of the field in the training set.

In [ ]:
lineno=348
sampleno=299

Do HANTS, first we have to extract the pixel data into the array datain. We use the Matlab function squeeze to do this, because NDVI is stored as a three dimensional array and we have to collect all values that belong to a single pixel. We use squeeze because NDVI(lineno,sampleno,:) does not return an array, but a list, and squeeze will convert this list to an array.

HANTS Outputs are:

      amp   = array of amplitudes, first element is the average of the curve
      phi   = array of phases, first element is zero
      dataout	= array holding reconstructed time series 


In [ ]:
from HANTS import *

datain = np.squeeze(NDVI[:, lineno, sampleno])
amp, phi, dataout = HANTS(ndates, nb, nf, datain, ts.transpose(), HiLo, low, high, fet, dod, delta)

### Plot the original vs reconstructed time series

In [ ]:
%matplotlib inline
plt.figure(figsize=(10,6))
plt.plot(ts[0,:], datain, 'b*', label='Original NDVI')
plt.plot(ts[0, :], dataout, 'r', label='Smoothed with HANTS')
plt.xlim([1, 365])
plt.ylim([0, 1])
plt.xlabel('Days from 12 April 2017')
plt.ylabel('NDVI [-]')
plt.title('NDVI - Sentinel 2, WA');
plt.legend()
plt.show()

## Apply HANTS for all pixels

Apply the HANTS algorithm for all pixels, and save the amplitude, phase and reconstucted NDVI.
<div class="alert alert-block alert-danger">
<b>THIS IS A VERY TIME CONSUMING PROCESS THAT MAY TAKE 15-30 MINUTES! SO ONLY DO THIS WHEN YOUR ARE SURE YOU GOT THE CORRECT PARAMETERS, OR TEST IT FIRST ON A SMALLER SAMPLE. </b>
</div>


In [ ]:
print('Smoothing and filling the missing data, please be patient ... ')
# Data is already dimensioned properly in Python, so don't need to do the following:
# To work conveniently with HANTS the data must be dimensioned [time,lat,lon]
NDVIp = NDVI
ni, ny, nx = NDVIp.shape

# Initialize new arrays to collect the HANTS output
NDVI_HANTS = np.zeros((ni, ny, nx))
amp = np.zeros((nf + 1, ny, nx))
phi = np.zeros((nf + 1, ny, nx))

# Call HANTS in a double loop, line by line (SEE THE WARNING)
for line in range(0, ny):
    # for line in range(0, 10): # use this for smaller sample
    print("Doing line %d out of %d \n" % (line, ny))
    for sample in range(0, nx):
        data = NDVIp[:, line, sample]
        if np.isnan(data).sum() != ni:
            data[np.isnan(data)] = low - 1.0
            amp[:, line, sample], phi[:, line, sample], NDVI_HANTS[:, line, sample] = HANTS(ndates, nb, nf, data,
                                                                                            ts.transpose(), HiLo,
                                                                                            low, high, fet, dod,
                                                                                            delta)

print("Done with smoothing and filling")

<div class='alert alert-info'>
Variables can be save and load using Spyder IDE. Make use of that for robust performance. HEre we will make use of pickle library to store the variables. 
</div>

In [ ]:
# Saving the variables
import pickle

f = open('NDVIr.pckl', 'wb')
pickle.dump([x, y, NDVI_HANTS, amp, phi], f)
f.close()

In [ ]:
# Loading the variables
# f = open('NDVIr.pckl', 'rb')
# x, y, NDVI_HANTS, amp, phi = pickle.load(f)
# f.close()

In [ ]:
# Writing NDVI file back
profile = src.profile
with rasterio.open('NDVI_reconstructed.tif', 'w', **profile) as dst:
    for id, layer in enumerate(NDVI_HANTS, 1):
        dst.write_band(id, NDVI_HANTS[id - 1, :, :])

# Writing Amplitude file back
profile = src.profile
with rasterio.open('NDVI_amplitude.tif', 'w', **profile) as dst:
    for id, layer in enumerate(amp, 1):
        dst.write_band(id, amp[id - 1, :, :])

# Writing NDVI file back
profile = src.profile
with rasterio.open('NDVI_phase.tif', 'w', **profile) as dst:
    for id, layer in enumerate(phi, 1):
        dst.write_band(id, phi[id - 1, :, :])

## Create a video of the reconstruced NDVI data

Create a video showing the reconstructed NDVI data.

In [ ]:
# Your code here

## Plot amplitude and phase from HANTS


In [ ]:
# Your code here

## Analyze the training dataset
The training dataset is a shapefile with information on the actual crop that is growing in the fields. The original training dataset was in UTM zone 29N. We used QGIS to convert this to UTM zone 30N, which was is also the zone that is used for the NDVI data.

First we must read the shape file with training data
```python
training = shapefile.Reader("Training_set/Training_2017_UTM30N_WGS84_SP.shp")
```

In [ ]:
# Your code here

The training dataset consists of 15 polygons. We will create an plot with the amplitude data and overlay the polygon boundaries.

In [ ]:
plt.figure(figsize=(12,8))
for shape in training.shapeRecords():
    x = [i[0]/1000 for i in shape.shape.points[:]]
    y = [i[1]/1000 for i in shape.shape.points[:]]
    plt.plot(x,y, color='yellow', linewidth=2)
greys = mpl.cm.get_cmap('Greys_r')
plt.imshow(amp[1, :, :], extent=extent, cmap=greys)
plt.axis('image')
plt.colorbar(label='Amplitude [-]')
plt.ylabel('North [km]')
plt.xlabel('East [km]')
plt.title('NDVI Training set')
plt.show()
plt.show()